# Hidden Unit Dynamics

### Encoder Networks:

<img src="images/encoder-network.png" width="50%">

The goal of this *encoder network* is to reconstruct the same input vector in the output layer after having passing through the bottleneck hidden layer, which forces the input vector's dimension to be reduced from 5 to 3.


### Hidden Unit Space:

The hidden unit space is a set of all possible hidden layer vectors. We can graphically represent the hidden unit space with coordinates of the hidden layer activations vector and lines representing the decision boundary formed by the hidden-output weights.

<img src="images/hidden-unit-space-star16.png" width="50%">

Consider a 5-2-5 encoder network, for example. 
- If we use a sigmoid activation, then the activation values are bound in the range $[0, 1]$
- If we have tanh activation, then all hidden unit activations are bound in the range $[-1, 1]$

The lines in the hidden unit space graph determines the output activation threshold. 

After the network is fully trained, every input vector will be mapped to a hidden unit activation vector that occupies its own decision region. With this, even though we have a bottleneck layer of dimension 2, all 5 1-hot encoded inputs can still be uniquely distinguished and reproduced at the output layer.

This idea of hidden unit activations being mapped to their own decision boundary generalises to higher dimensions, if we had more than 2 hidden units.

### Weight Space Symmetry:

- Swapping the positions of any pair of hidden nodes does not affect the overall network performance
- Reversing the sign of all incoming and outgoing weights in any hidden node won't affect overall network performance, assuming the use of a symmetric transfer function like $\tanh$
- Hidden nodes tend to have similar roles in affecting the output prediction initially, then gradually specialise as training progresses

The first two points above gives rise to a very large number of other possible weight configurations that would all perform equally well. Eg. suppose there are 10 nodes in a hidden layer. There are $10!$ ways to permute them, plus $2^{10}$ additional different permutations for the positive/negative signs for the weights coming in and out of each node. This gives $10!\cdot 2^{10}$ other possible configurations in the possible weight space that would all perform equally well.


### Controlled Nonlinearity:

- For smaller weights, each layer implements an approximately linear activation function, so multiple layers also implement an approximately linear activation function
    - This means that the stacking of multiple layers would be almost equivalent to a single layer network (no matter how many layers we have, if all outputs are linear, the final output is nothing more than a linear function of the first layer's input)
- For larger weights, the activation function approximates a step function which makes learning very slow - because the gradient is approximately 0, so gradient descent causes very small updates in weight
    - Just switching from a sigmoid activation function to the ReLU activation function can make gradient descent work much faster
- For typical weights, the activation function is close to linear, but takes advantage of a limited degree of nonlinearity

<img src="images/tanhx-and-x.png" width="50%"/>



### Vanishing/Exploding Gradient Problem: 

- __*Vanishing gradient problem*__ &mdash; when the gradients diminish exponentially across each layer from output to input
    - Stacking more layers means the weight updates are concentrated on the layers closer to the output layer rather than the layers closer to the input layer.
    - Remember in backpropagation, we rely on a product of partial differentials to compute the derivative of the error function with respect to a particular weight parameter. As we go back to earlier layers, this product involves more and more terms. If each of those terms are $<1$, then the overall product is tiny, so applying gradient descent at earlier nodes barely changes anything. 
    - The weights at the earlier layers effectively get 'stuck' and don't end up contributing much towards the network's minimisation of the error function.
- __*Exploding gradient problem*__ &mdash; when the gradients grow exponentially across each layer from output to input
    - By similar reasoning, if the partial differentials passed back to earlier layers are $>1$, then the gradients computed at the earlier layers will be large and will not properly converge to optimal weight values.


#### Avoiding vanishing/exploding gradient:
There are some standard ways for combating the vanishing/exploding gradient problem:
- <a href="https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79">*Weight initialisation*</a> &mdash; a set of techniques for setting initial weight values that tend to prevent *activations* from vanishing/exploding in the feed-forward pass. This then prevents the gradients from vanishing/exploding in the backward-pass
- <a href="https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/">*Batch normalisation*</a> &mdash; normalising the input vector to the layers of the neural network by scaling and centering the components to a reasonable range.
- *Layerwise unsupervised pre-training* &mdash; involves first training an autoencoder network to achieve a more reasonable weight configuration prior to the training for the original task. This is similar to weight initialisation  
- Using different activation functions like $\texttt{ReLU}$ or $\texttt{SeLU}$:
    <table width="50%">
      <tr>
        <td> <img src="images/relu.png"></td>
        <td><img src="images/selu.png"></td>
       </tr>
    </table>


### Combating Overfitting:
The following are a few standard methods for combating overfitting in neural networks:
- __*Early stopping*__ — stopping the training when the neural network has achieved a minimal error on the testing or validation dataset 
    <table>
        <tr>
            <td> 
                <img src="images/error-vs-weight-updates-1.png"/>
            </td>
            <td>
                <img src="images/error-vs-weight-updates-2.png"/>
            </td>
        </tr>
    </table>
    Experimenting with the number of hidden nodes and number of weight updates and plotting graphs like this helps us decide a good number of training epochs to stop training at. In general, the more epochs, the greater the chance of overfitting to the training data.
- __*Dropout*__ — randomly choosing a subset of nodes to not be used for each mini- batch. Each node is chosen to be 'dropped' from training with a fixed probability $\pi$ you choose
    - When the network is deployed, all nodes are active, but all the activation functions are multiplied by a factor $1 -\pi$. This is so that the overall contribution of each node in total is equal on average to the total contribution when the network had dropped out nodes
    - Inspired by biological neural networks where neurons may 'drop out' and other neurons have to 'fill' in for their contribution to the output
    
- __*Ensembling*__ — training multiple different networks on the same task and then making a prediction as a majority vote across the ensemble of networks.
    - Bagging — each network can be trained on the same 'bag' of datapoints with replacement. For one network, some items will be trained on multiple times or none at all, and this will be different for each network, giving rise to diversity across each network 
    - Dropout is implicitly a form of ensembling because on deployment, it's like the combination of multiple different architectures

### Resources:
- <a href="https://towardsdatascience.com/the-vanishing-exploding-gradient-problem-in-deep-neural-networks-191358470c11">Vanishing/Exploding gradient problem</a>

- <a href="https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79">Weight Initialisation</a>
- <a href="https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/">Batch normalisation</a>
- <a href="https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/">Layerwise Unsupervised Pretraining</a>
